In [1]:
!cat /proc/meminfo

MemTotal:       65951648 kB
MemFree:        54120168 kB
MemAvailable:   55665436 kB
Buffers:          656600 kB
Cached:          1299456 kB
SwapCached:            0 kB
Active:         10430724 kB
Inactive:         866712 kB
Active(anon):    9341392 kB
Inactive(anon):       60 kB
Active(file):    1089332 kB
Inactive(file):   866652 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                72 kB
Writeback:             0 kB
AnonPages:       9341588 kB
Mapped:           177588 kB
Shmem:                72 kB
Slab:             356564 kB
SReclaimable:     314180 kB
SUnreclaim:        42384 kB
KernelStack:        8480 kB
PageTables:        24632 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    32975824 kB
Committed_AS:   12228832 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
!pip install xgboost msgpack

    100% |████████████████████████████████| 16.6MB 3.1MB/s eta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import io
import gc
import boto3
import re

# custom functions and objects
from utils import *
# from utils.sample_strategies import stg2_sample, stg2_period_samples, stg2_sample_exclude, rfc_key_features_data

# ruta de trabajo en s3
bucket = 'boi-banregio'
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV'

# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)
seed = 0

In [4]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/JAT_MCV_VAR_VARIABLES_SELECCION4.csv"
data_file_key = "{}/{}".format(prefix, data_file_name)
data_obj = s3_bucket_resource.Object(data_file_key).get()
data = pd.read_csv(io.BytesIO(data_obj['Body'].read()))

In [7]:
data.iloc[:,820:1020].describe()

,NUM_CLAVOBS_BK_CF_CL_ACC,NUM_CLAVOBS_BK_CF_CL_ACC_03M,NUM_CLAVOBS_BK_CF_CL_ACC_06M,NUM_CLAVOBS_BK_CF_CL_ACC_12M,NUM_CLAVOBS_BK_CL_ACC,NUM_CLAVOBS_BK_CL_ACC_03M,NUM_CLAVOBS_BK_CL_ACC_06M,NUM_CLAVOBS_BK_CL_ACC_12M,NUM_CLAVOBS_BK_CS_CL_ACC,NUM_CLAVOBS_BK_CS_CL_ACC_03M,...,NUM_CLAVOBS_UN_CRE_OTHER_CL_ACC_06M,NUM_CLAVOBS_UN_CRE_OTHER_CL_ACC_12M,NUM_CLAVOBS_UN_CRE_PQ_CL_ACC,NUM_CLAVOBS_UN_CRE_PQ_CL_ACC_03M,NUM_CLAVOBS_UN_CRE_PQ_CL_ACC_06M,NUM_CLAVOBS_UN_CRE_PQ_CL_ACC_12M,NUM_CLAVOBS_UN_CRE_R_CL_ACC,NUM_CLAVOBS_UN_CRE_R_CL_ACC_03M,NUM_CLAVOBS_UN_CRE_R_CL_ACC_06M,NUM_CLAVOBS_UN_CRE_R_CL_ACC_12M
count,424082.0,424082.0,424082.0,424082.0,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,...,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000,424082.000000
mean,0.0,0.0,0.0,0.0,1.272353,1.272353,2.570413,5.200949,0.912614,0.912614,...,0.000141,0.000141,0.002474,0.002474,0.005119,0.010446,0.000127,0.000127,0.000328,0.000672
std,0.0,0.0,0.0,0.0,20.489835,20.489835,41.556488,87.752349,15.141086,15.141086,...,0.038359,0.038359,0.088379,0.088379,0.165444,0.306102,0.011894,0.011894,0.022722,0.039712
min,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,0.0,0.0,0.0,3723.000000,3723.000000,6547.000000,11559.000000,3721.000000,3721.000000,...,12.000000,12.000000,8.000000,8.000000,15.000000,25.000000,2.000000,2.000000,4.000000,6.000000


## Estrategia 2

In [8]:
f_str = io.StringIO()
sample_data = data[stg2_sample]
sample_data.to_csv(f_str, index=False)
sample_data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY2.csv"
sample_data_obj_key =  "{}/{}".format(prefix, sample_data_file_name)
s3_bucket_resource.Object(sample_data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': 'FDF18C187684EBAF',
  'HostId': 'rZnFslR608dxjDiD0cZy3JfmjRvTjElPnBHttc5E8mWCvYflL+Z8MIPpzWOBEio7zGmQE9kduVE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'rZnFslR608dxjDiD0cZy3JfmjRvTjElPnBHttc5E8mWCvYflL+Z8MIPpzWOBEio7zGmQE9kduVE=',
   'x-amz-request-id': 'FDF18C187684EBAF',
   'date': 'Tue, 06 Nov 2018 23:41:47 GMT',
   'etag': '"3e813cd79fcb3d412b815dc2ddb7f3e5"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"3e813cd79fcb3d412b815dc2ddb7f3e5"'}

## Estrategia 1

In [9]:
f_str = io.StringIO()
sample_data = data[stg1_sample]
sample_data.to_csv(f_str, index=False)
sample_data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/STRATEGY1.csv"
sample_data_obj_key =  "{}/{}".format(prefix, sample_data_file_name)
s3_bucket_resource.Object(sample_data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': '342175C607D059E8',
  'HostId': 'Ysd172i7jZzeUxQCYC4u0bTZXZytN3p7XtGGXSxAMCCMQxCY6r4f+b81PVljKfLsNHaj0aPGj8c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ysd172i7jZzeUxQCYC4u0bTZXZytN3p7XtGGXSxAMCCMQxCY6r4f+b81PVljKfLsNHaj0aPGj8c=',
   'x-amz-request-id': '342175C607D059E8',
   'date': 'Tue, 06 Nov 2018 23:45:18 GMT',
   'etag': '"80057d8ab276b9e575361010625a7d63"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"80057d8ab276b9e575361010625a7d63"'}

In [11]:
stg1_sample.sum()
#sample_data.shape

65753

## Validation Sample

In [8]:
validation = rfc_key_features_data['FECHA'] > 201705

In [9]:
validation.sum()

82377

In [10]:
f_str = io.StringIO()
sample_data = data[validation]
sample_data.to_csv(f_str, index=False)
sample_data_file_name = "DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/SAMPLES/VALIDATION+201705.csv"
sample_data_obj_key =  "{}/{}".format(prefix, sample_data_file_name)
s3_bucket_resource.Object(sample_data_obj_key).put(Body=f_str.getvalue())

{'ResponseMetadata': {'RequestId': '10E70A3B008076A4',
  'HostId': 'bYuWi5OnJvK/EPYIcZsu77oBNoxGgTFcOOneuWvngGqLz9ziP5SZcpsQL8O1/MOZ9vRecOCqhuQ=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bYuWi5OnJvK/EPYIcZsu77oBNoxGgTFcOOneuWvngGqLz9ziP5SZcpsQL8O1/MOZ9vRecOCqhuQ=',
   'x-amz-request-id': '10E70A3B008076A4',
   'date': 'Tue, 06 Nov 2018 16:51:43 GMT',
   'etag': '"23c764d1a30352d933352aba5cb13170"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"23c764d1a30352d933352aba5cb13170"'}